In [5]:
import scipy.sparse
import numpy as np
import miniQM as mqm

# Tutorial for miniQM

miniQM is a library for the evaluation of the ground state and the propagation of simple model systems in second quantization.\
It starts by defining first a Hilbert space (a set of basis elements and rules to operate on them) and then uses the rules to transform a list of operators into matrices (generally in compressed sparse row form).

## Defining a Hilbert space 

To define a basic Hilbert space in miniQM one has to first choose if it is fermionic or bosonic space. Let us say we are working with fermions. For example we consider 2 spinless fermions on 5 sites. Here the notion of "site" can represent any notion of spin-orbital that can be empty or occupied in the Slater determinant or simply actual Hubbard sites. The definition of the Hilbert space (hs) looks like:

In [6]:
nsites = 5
npart = 2
hs = mqm.BasicFermions.firstdet(npart, nsites)

Why is invoking the firstdet method ? Well, in miniQM there is no difference between an element (here a slater determinant) of a Hilbert space and the Hilbert space itself. This is due to the fact that each element of a Hilbert space comes with the set of rules that applies to it. We generally define a fermionic Hilbert space by its first determinant + the rules. And indeed if we print it...

In [7]:
print(hs)

[1, 1, 0, 0, 0]


We obtain the array of 0 and 1s representing the occupation of each site of the first determinant $\vert 1 1 0 0 \rangle$ (order arbitrarily decided when implementing miniQM). It is also possible to define the Hilbert space using any other determinant. 

In [20]:
hs = mqm.BasicFermions([0, 0, 1, 1, 0])
print(hs)

[0, 0, 1, 1, 0]


The class BasicFermions possesses its own range method, which can be useful to see the list of basis elements in the Hilbert space and how they are ordered. Here for example:

In [21]:
for elem in hs.range():
    print(elem)

[1, 1, 0, 0, 0]
[1, 0, 1, 0, 0]
[1, 0, 0, 1, 0]
[1, 0, 0, 0, 1]
[0, 1, 1, 0, 0]
[0, 1, 0, 1, 0]
[0, 1, 0, 0, 1]
[0, 0, 1, 1, 0]
[0, 0, 1, 0, 1]
[0, 0, 0, 1, 1]


One can access the index of a given determinant using the index attribute. We will copy the determinant we use for hs and look at his index 

In [22]:
elem1 = hs.copy()
print("index:", elem1.index, "element:", elem1)

index: 7 element: [0, 0, 1, 1, 0]


We can verify this is indeed the 7th element (starting at 0) of the previous list.\
In this list one can that notice that the number of particles is conserved. This is the default behavior of BasicFermions. It is possible to define a larger Hilbert space where the number of particles is not conserved. In this case the number of particles can be set to any value as it irrelevant and firstdet is always [0, 0, .., 0]. We will show it on a system with only 4 sites as the Hilbert space got bigger.

In [23]:
hs = mqm.BasicFermions.firstdet(npart, 3, nconserved=False)
# or as explained previously
hs = mqm.BasicFermions([0, 1, 0, 1], nconserved=False)

for elem in hs.range():
    print(elem.index, ":", elem)


0 : [0, 0, 0, 0]
1 : [1, 0, 0, 0]
2 : [0, 1, 0, 0]
3 : [1, 1, 0, 0]
4 : [0, 0, 1, 0]
5 : [1, 0, 1, 0]
6 : [0, 1, 1, 0]
7 : [1, 1, 1, 0]
8 : [0, 0, 0, 1]
9 : [1, 0, 0, 1]
10 : [0, 1, 0, 1]
11 : [1, 1, 0, 1]
12 : [0, 0, 1, 1]
13 : [1, 0, 1, 1]
14 : [0, 1, 1, 1]
15 : [1, 1, 1, 1]


## Product of Hilbert spaces

In miniQM it is possible to define a product of Hilbert spaces. For example if we consider a Hubbard dimer with two particles of different spin on two sites. It can be represented as the product of one fermionic Hilbert space for spin up with the same for spin down.

In [24]:
hs_up = mqm.BasicFermions.firstdet(1,2)
hs_down = mqm.BasicFermions.firstdet(1,2)
hs = hs_up*hs_down
for elem in hs.range():
    print(elem.index, ":", elem)

0 : [[1, 0], [1, 0]]
1 : [[1, 0], [0, 1]]
2 : [[0, 1], [1, 0]]
3 : [[0, 1], [0, 1]]


which, if we say the first Hilbert space of the product represents spin up, is to be interpreted as \
0 : $\vert \uparrow \downarrow, 0 \rangle = \vert 1, 0 \rangle_\uparrow \vert 1, 0 \rangle_\downarrow$ \
1 : $\vert \uparrow , \downarrow \rangle = \vert 1, 0 \rangle_\uparrow \vert 0, 1 \rangle_\downarrow$ \
2 : $\vert  \downarrow, \uparrow \rangle = \vert 0, 1 \rangle_\uparrow \vert 1, 0 \rangle_\downarrow$ \
3 : $\vert 0, \uparrow \downarrow \rangle = \vert 0, 1 \rangle_\uparrow \vert 0, 1 \rangle_\downarrow$

Let us continue to work with a Hubbard dimer for the next part.

## Operators

In miniQM it is possible to define creation/annihilation operators and make them act on an element. An creation/annihilation operator is a simple list of the for [ispace, isite, +-1] where ispace is the index of the Hilbert space we act on, isite the index of the site, and we use +1 for creation and -1 for annihilation operator. For example in the Hubbard dimer we have and Hilbert space 0 (spin up) with site 0 and site 1 and Hilbert space 1 (spin down) with site 0 and site 1. The creation operator $a^{\dagger}_{0\downarrow}$ is therefore [1, 0, +1] and $a_{0\uparrow}$ is [0, 0, -1], etc.. Let us see how we act on an element

In [25]:
elem = hs.copy()
print("The element is the first one of the Hilbert space", elem)
op = [0, 0, -1]
result = elem.operator(op)
print(result[0], result[1])
elem_res = result[0]

The element is the first one of the Hilbert space [[1, 0], [1, 0]]
[[0, 0], [1, 0]] -1


We did the following  
$$ a_{0\uparrow} \vert 1, 0 \rangle_\uparrow \vert 1, 0 \rangle_\downarrow = \alpha \vert 0, 0 \rangle_\uparrow \vert 1, 0 \rangle_\downarrow $$
and the method operator returns the couple $ (\vert 0, 0 \rangle_\uparrow \vert 1, 0 \rangle_\downarrow,~ \alpha) $ with $\alpha = -1$ because of the ordering of operators. Because our operation did not preserve the number of particles, the element returned is outside of our previously defined Hilbert space:

In [26]:
for elem in elem_res.range():
    print(elem.index, ":", elem)

0 : [[0, 0], [1, 0]]
1 : [[0, 0], [0, 1]]


Generally we will never us the operator method. For example, if we want to implement a hoping term $\hat{T}$ in our Hubbard dimer, the operator for the spin up looks like
$$ \hat{T}_{\uparrow}=-ta_{1\uparrow}^{\dagger}a_{0\uparrow}-ta_{0\uparrow}^{\dagger}a_{1\uparrow} $$
In miniQM this is kind of operator is a list. We decompose how we construct it the first step is to write
$$ tterm\_up = [[-t,[a_{1\uparrow}^{\dagger},a_{0\uparrow}]],[-t,[a_{0\uparrow}^{\dagger}a_{1\uparrow}]]] $$
Now we can replace the terms like $a_{1\uparrow}^{\dagger}$ by [0, 1, +1] as defined with the previous convention.
So in code (choosing $t=0.5$) we obtain 

In [27]:
tval = 0.5
tterm_up = []
tterm_up += [[-tval,[[0, 1, +1],[0, 0, -1]]]]
tterm_up += [[-tval,[[0, 0, +1],[0, 1, -1]]]]
print(tterm_up)

[[-0.5, [[0, 1, 1], [0, 0, -1]]], [-0.5, [[0, 0, 1], [0, 1, -1]]]]


This is also equivalent to use the function tterm from miniQM

In [28]:
ispace = 0
ipos1 = 1
ipos2 = 0
tterm_up = mqm.tterm(tval, ispace, ipos1, ipos2)
print(tterm_up)

[[0.5, [[0, 1, 1], [0, 0, -1]]], [0.5, [[0, 0, 1], [0, 1, -1]]]]


In [29]:
tterm_down = []
tterm_down += [[-tval,[[1, 1, +1],[1, 0, -1]]]]
tterm_down += [[-tval,[[1, 0, +1],[1, 1, -1]]]]
tterm = tterm_up + tterm_down

We used the concatenation operator +. Now we just have to transform this into a matrix in the basis of this Hilbert space (see hs.range() to know the basis). The method oplist_to_csr of miniQM provides a convenient way to do that.b The output is csr matrix from scipy which allows faster matrix multiplications, but here to display it we will convert it into a numpy array using the method toarray.

In [30]:
tterm_csr = mqm.oplist_to_csr(tterm, hs)
tterm_mat = tterm_csr.toarray()
print(tterm_mat)

[[ 0. +0.j -0.5+0.j  0.5+0.j  0. +0.j]
 [-0.5+0.j  0. +0.j  0. +0.j  0.5+0.j]
 [ 0.5+0.j  0. +0.j  0. +0.j -0.5+0.j]
 [ 0. +0.j  0.5+0.j -0.5+0.j  0. +0.j]]


One can check this is indeed the matrix of the hoping operator in the basis:

In [31]:
for elem in hs.range():
    print(elem.index, ":", elem)

0 : [[1, 0], [1, 0]]
1 : [[1, 0], [0, 1]]
2 : [[0, 1], [1, 0]]
3 : [[0, 1], [0, 1]]


Now for the U term of the Hubbard 
$$ \hat{U}=U\hat{n}_{0\uparrow}\hat{n}_{0\downarrow}+U\hat{n}_{1\uparrow}\hat{n}_{1\downarrow} $$
we get

In [32]:
Uval = 10
uterm = []
uterm += [[Uval, [[0, 0, 1], [0, 0, -1], [1, 0, 1], [1, 0, -1]]]]
uterm += [[Uval, [[0, 1, 1], [0, 0, -1], [1, 1, 1], [1, 1, -1]]]]
print(uterm)

[[10, [[0, 0, 1], [0, 0, -1], [1, 0, 1], [1, 0, -1]]], [10, [[0, 1, 1], [0, 0, -1], [1, 1, 1], [1, 1, -1]]]]


In [33]:
ispace1 = 0
ispace2 = 1
ipos = 0
uterm = mqm.uterm(Uval, ispace1, ispace2, ipos)
ipos = 1
uterm += mqm.uterm(Uval, ispace1, ispace2, ipos)
print(uterm) #equivalent to precedent up to a commutation between operators

[[-10, [[0, 0, 1], [1, 0, 1], [0, 0, -1], [1, 0, -1]]], [-10, [[0, 1, 1], [1, 1, 1], [0, 1, -1], [1, 1, -1]]]]


The full hamiltonian for the symmetric Hubbard dimer is obtained by summing the two terms:

In [34]:
h_list = tterm + uterm
h_csr = mqm.oplist_to_csr(h_list, hs)
h_csr = h_csr.real #not necessary, but as we work we real numbers here...
h_mat = h_csr.toarray()
print(h_mat)

[[10.  -0.5  0.5  0. ]
 [-0.5  0.   0.   0.5]
 [ 0.5  0.   0.  -0.5]
 [ 0.   0.5 -0.5 10. ]]


We can of course compute the eigenvectors (see scipy/numpy documentation):

In [40]:
eigvals, eigvecs = scipy.sparse.linalg.eigsh(h_csr, k=3)
# print the groundstate
print("SCIPY")
print("GS energy:",eigvals[0], "|| GS vec:", eigvecs[:,0])
print("======================")
eigvals, eigvecs = np.linalg.eigh(h_mat)
print("NUMPY")
print("GS energy:",eigvals[0], "|| GS vec:", eigvecs[:,0])

gsvec = eigvecs[:,0]

SCIPY
GS energy: -0.09901951359278353 || GS vec: [ 0.06967662  0.70366552 -0.70366552 -0.06967662]
NUMPY
GS energy: -0.09901951359278635 || GS vec: [ 0.06967662  0.70366552 -0.70366552 -0.06967662]
